In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['NP', 'NPs']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('nanoparticles', 4903),
 ('nucleus pulposus', 1029),
 ('nucleoprotein', 990),
 ('nanoparticle', 788),
 ('nonylphenol', 391),
 ('neuropathic pain', 154),
 ('nucleocapsid protein', 125),
 ('natriuretic peptide', 116),
 ('nitroprusside', 86),
 ('nasal polyps', 64),
 ('nasopharyngeal', 58),
 ('nitrophenyl acetyl', 55),
 ('nitrophenylacetyl', 31),
 ('neuropsychiatric', 31),
 ('neuropsychological', 30),
 ('nasal polyposis', 28),
 ('neck pain', 25),
 ('neural progenitor', 24),
 ('neuritic plaque', 22),
 ('normal pregnant', 21),
 ('nurse practitioner', 20),
 ('nosocomial pneumonia', 20),
 ('non pregnant', 18),
 ('nitrophenol', 17),
 ('np', 16),
 ('nuclear protein', 15),
 ('nucleoplasmin', 15),
 ('neural precursors', 15),
 ('negative priming', 14),
 ('nonpreferring', 13),
 ('nasopharynx', 13),
 ('normal protein', 13),
 ('nitrophenyl', 12),
 ('neurophysin', 12),
 ('neopterin', 11),
 ('neuropoietin', 11),
 ('nucleoside phosphorylase', 10),
 ('non preferring', 10),
 ('neural precursor', 10),
 ('

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [7]:
names

{'MESH:D000070614': 'Nucleus Pulposus',
 'MESH:D009698': 'Nucleoproteins',
 'CHEBI:CHEBI:50803': 'nanoparticle',
 'CHEBI:CHEBI:79516': 'Nonylphenol',
 'MESH:D009437': 'Neuralgia',
 'MESH:D019590': 'Nucleocapsid Proteins',
 'FPLX:Natriuretic_peptide': 'Natriuretic Peptide',
 'CHEBI:CHEBI:7596': 'nitroprusside',
 'MESH:D009298': 'Nasal Polyps',
 'MESH:D019547': 'Neck Pain',
 'MESH:D000077299': 'Healthcare-Associated Pneumonia',
 'CHEBI:CHEBI:25562': 'nitrophenol',
 'MESH:D056489': 'Nucleoplasmins',
 'MESH:D009305': 'Nasopharynx',
 'CHEBI:CHEBI:36080': 'protein',
 'CHEBI:CHEBI:28670': 'neopterin',
 'EFO:0001651': 'precursor',
 'MESH:D019251': 'Nucleocapsid',
 'CHEBI:CHEBI:7476': 'naproxen',
 'MESH:D054258': 'Neural Plate',
 'MESH:D011312': 'Pressure',
 'MESH:D020494': 'Phrases',
 'MESH:D011677': 'Punctures',
 'MESH:D000066829': 'Neuroprotection',
 'MESH:D009479': 'Neuropeptides',
 'MESH:D010336': 'Pathology',
 'GO:GO:0016310': 'phosphorylation',
 'MESH:D011247': 'Pregnancy',
 'MESH:D00007

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-10-31 04:40:56] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'n phenylmaleimide': 'MESH:C028525',
  'n protein': 'ungrounded',
  'nano particles': 'ungrounded',
  'nano size particles': 'ungrounded',
  'nanoparticle': 'MESH:D053758',
  'nanoparticles': 'MESH:D053758',
  'nanoparticles delivery systems': 'MESH:D053758',
  'nanoparticles s': 'MESH:D053758',
  'nanoparticulate': 'MESH:D053758',
  'nanopaticles': 'MESH:D053758',
  'nanoplastics': 'ungrounded',
  'nanoplatinum': 'ungrounded',
  'nanoporous': 'ungrounded',
  'nanoprobes': 'ungrounded',
  'napabucasin': 'MESH:C000621033',
  'naphthalene': 'CHEBI:CHEBI:16482',
  'naphthyl piperazine': 'ungrounded',
  'naproxen': 'CHEBI:CHEBI:7476',
  'nasal passages': 'ungrounded',
  'nasal polyposis': 'ungrounded',
  'nasal polyps': 'ungrounded',
  'nasopharyngeal': 'MESH:D009305',
  'nasopharynx': 'MESH:D009305',
  'natriuretic peptide': 'FPLX:Natriuretic_peptide',
  'natural production': 'ungrounded',
  'natural raw shelled pistachios': 'ungrounded',
  'neck pain': 'MESH:D019547',
  'neck pressure'

In [27]:
grounding_map, names, pos_labels = [{'n phenylmaleimide': 'MESH:C028525',
  'n protein': 'ungrounded',
  'nano particles': 'ungrounded',
  'nano size particles': 'ungrounded',
  'nanoparticle': 'MESH:D053758',
  'nanoparticles': 'MESH:D053758',
  'nanoparticles delivery systems': 'MESH:D053758',
  'nanoparticles s': 'MESH:D053758',
  'nanoparticulate': 'MESH:D053758',
  'nanopaticles': 'MESH:D053758',
  'nanoplastics': 'ungrounded',
  'nanoplatinum': 'ungrounded',
  'nanoporous': 'ungrounded',
  'nanoprobes': 'ungrounded',
  'napabucasin': 'MESH:C000621033',
  'naphthalene': 'CHEBI:CHEBI:16482',
  'naphthyl piperazine': 'ungrounded',
  'naproxen': 'CHEBI:CHEBI:7476',
  'nasal passages': 'ungrounded',
  'nasal polyposis': 'ungrounded',
  'nasal polyps': 'ungrounded',
  'nasopharyngeal': 'MESH:D009305',
  'nasopharynx': 'MESH:D009305',
  'natriuretic peptide': 'FPLX:Natriuretic_peptide',
  'natural production': 'ungrounded',
  'natural raw shelled pistachios': 'ungrounded',
  'neck pain': 'MESH:D019547',
  'neck pressure': 'ungrounded',
  'necrotizing pancreatitis': 'ungrounded',
  'necrotizing pneumonia': 'MESH:D000071067',
  'nedaplatin': 'CHEBI:CHEBI:31898',
  'needle puncture': 'ungrounded',
  'negative pressure': 'ungrounded',
  'negative priming': 'ungrounded',
  'negative visuospatial priming': 'ungrounded',
  'neglect patients': 'ungrounded',
  'neopterin': 'CHEBI:CHEBI:28670',
  'nepeta pogonosperma': 'ungrounded',
  'nereu programmed': 'ungrounded',
  'neural plate': 'ungrounded',
  'neural precursor': 'ungrounded',
  'neural precursors': 'ungrounded',
  'neural progenitor': 'ungrounded',
  'neuritic plaque': 'ungrounded',
  'neuronal precursors': 'ungrounded',
  'neuropathic pain': 'MESH:D009437',
  'neuropeptide': 'MESH:D009479',
  'neurophysin': 'ungrounded',
  'neuropilin': 'MESH:D039921',
  'neuropoietin': 'HGNC:33173',
  'neuroprogenitor': 'ungrounded',
  'neuroprogenitors': 'ungrounded',
  'neuroprostanes': 'MESH:D058632',
  'neuroprotection': 'MESH:D000066829',
  'neuropsychiatric': 'ungrounded',
  'neuropsychological': 'ungrounded',
  'neuropsychological performed': 'ungrounded',
  'neutral potential': 'ungrounded',
  'neutral protease': 'ungrounded',
  'neutrophilic': 'ungrounded',
  'new physical': 'ungrounded',
  'ngf pacap': 'HGNC:241',
  'nifedipine': 'CHEBI:CHEBI:7565',
  'nitro phenacetyl': 'ungrounded',
  'nitro phenyl acetyl': 'ungrounded',
  'nitro phenylacetyl': 'ungrounded',
  'nitrophenacetyl': 'ungrounded',
  'nitrophenol': 'ungrounded',
  'nitrophenyl': 'ungrounded',
  'nitrophenyl acetyl': 'ungrounded',
  'nitrophenylacetyl': 'ungrounded',
  'nitroprusside': 'CHEBI:CHEBI:7596',
  'nitropyrene': 'ungrounded',
  'no pathology': 'ungrounded',
  'nocturnal polyuria': 'MESH:D011141',
  'node positive': 'ungrounded',
  'non parasitized': 'ungrounded',
  'non pathogenic': 'ungrounded',
  'non phosphorylation': 'ungrounded',
  'non preferring': 'ungrounded',
  'non pregnant': 'ungrounded',
  'non purified chow': 'ungrounded',
  'nonapeptide fragment': 'ungrounded',
  'nonperiodized': 'ungrounded',
  'nonpreferring': 'ungrounded',
  'nonpregnant': 'ungrounded',
  'nonyl phenol': 'CHEBI:CHEBI:79516',
  'nonylphenol': 'CHEBI:CHEBI:79516',
  'normal phase': 'ungrounded',
  'normal plasma': 'ungrounded',
  'normal pregnancy': 'ungrounded',
  'normal pregnant': 'ungrounded',
  'northern plains': 'ungrounded',
  'nosocomial pneumonia': 'ungrounded',
  'notalgia paresthetica': 'ungrounded',
  'noun phrase': 'ungrounded',
  'np': 'ungrounded',
  'np protein': 'ungrounded',
  'nucellar projection': 'ungrounded',
  'nuclear periphery': 'GO:GO:0034399',
  'nuclear protein': 'MESH:D009687',
  'nucleo protein': 'MESH:D009698',
  'nucleocapsid': 'MESH:D019251',
  'nucleocapsid protein': 'MESH:D019590',
  'nucleolus predominant': 'ungrounded',
  'nucleoplasmin': 'MESH:D056489',
  'nucleoprotein': 'MESH:D009698',
  'nucleoside phosphorylase': 'HGNC:7892',
  'nucleus pulposus': 'MESH:D000070614',
  'nucleus pulpous': 'MESH:D000070614',
  'nurse practitioner': 'ungrounded',
  'nutrient patterns': 'ungrounded',
  'symptom free sf nausea only': 'ungrounded'},
 {'MESH:C028525': 'N-phenylmaleimide',
  'MESH:D053758': 'Nanoparticles',
  'MESH:C000621033': 'napabucasin',
  'CHEBI:CHEBI:16482': 'naphthalene',
  'CHEBI:CHEBI:7476': 'naproxen',
  'MESH:D009305': 'Nasopharynx',
  'FPLX:Natriuretic_peptide': 'Natriuretic Peptide',
  'MESH:D019547': 'Neck Pain',
  'MESH:D000071067': 'Pneumonia, Necrotizing',
  'CHEBI:CHEBI:31898': 'nedaplatin',
  'CHEBI:CHEBI:28670': 'neopterin',
  'MESH:D009437': 'Neuralgia',
  'MESH:D009479': 'Neuropeptides',
  'MESH:D039921': 'Neuropilins',
  'HGNC:33173': 'CTF2P',
  'MESH:D058632': 'Neuroprostanes',
  'MESH:D000066829': 'Neuroprotection',
  'HGNC:241': 'ADCYAP1',
  'CHEBI:CHEBI:7565': 'nifedipine',
  'CHEBI:CHEBI:7596': 'nitroprusside',
  'MESH:D011141': 'Polyuria',
  'CHEBI:CHEBI:79516': 'Nonylphenol',
  'GO:GO:0034399': 'nuclear periphery',
  'MESH:D009687': 'Nuclear Proteins',
  'MESH:D019251': 'Nucleocapsid',
  'MESH:D019590': 'Nucleocapsid Proteins',
  'MESH:D056489': 'Nucleoplasmins',
  'MESH:D009698': 'Nucleoproteins',
  'HGNC:7892': 'PNP',
  'MESH:D000070614': 'Nucleus Pulposus'},
 ['CHEBI:CHEBI:7596',
  'CHEBI:CHEBI:79516',
  'FPLX:Natriuretic_peptide',
  'HGNC:33173',
  'MESH:D009437',
  'MESH:D009698',
  'MESH:D019590']]

In [28]:
excluded_longforms = ['np', 'n protein', 'np protein']

In [29]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [15]:
additional_entities = {}

In [16]:
unambiguous_agent_texts = {}

In [30]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [18]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [19]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[]

In [21]:
intersection2

[]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [31]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-31 05:08:31] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-31 05:15:43] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.8544828514288735 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [32]:
classifier.stats

{'label_distribution': {'ungrounded': 505,
  'CHEBI:CHEBI:7596': 81,
  'CHEBI:CHEBI:28670': 7,
  'MESH:D009698': 670,
  'MESH:D019590': 95,
  'CHEBI:CHEBI:7476': 6,
  'MESH:D009305': 46,
  'HGNC:7892': 5,
  'CHEBI:CHEBI:16482': 3,
  'MESH:D009687': 11,
  'MESH:D053758': 4826,
  'FPLX:Natriuretic_peptide': 126,
  'CHEBI:CHEBI:79516': 291,
  'MESH:D019251': 8,
  'MESH:D009479': 5,
  'MESH:D000070614': 599,
  'CHEBI:CHEBI:7565': 3,
  'MESH:D056489': 9,
  'HGNC:33173': 10,
  'MESH:D019547': 14,
  'CHEBI:CHEBI:31898': 3,
  'MESH:D009437': 105,
  'MESH:D039921': 4,
  'MESH:D000071067': 3,
  'MESH:C028525': 2,
  'HGNC:241': 1,
  'MESH:D011141': 3,
  'GO:GO:0034399': 2,
  'MESH:D000066829': 2,
  'MESH:C000621033': 2,
  'MESH:D058632': 2},
 'f1': {'mean': 0.854483, 'std': 0.006426},
 'precision': {'mean': 0.871979, 'std': 0.014218},
 'recall': {'mean': 0.845438, 'std': 0.014864},
 'MESH:D000070614': {'f1': {'mean': 0.983253, 'std': 0.004514},
  'pr': {'mean': 0.979944, 'std': 0.012532},
  'rc':

In [33]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [34]:
disamb.dump(model_name, results_path)

In [35]:
print(disamb.info())

Disambiguation model for NP, and NPs

Produces the disambiguations:
	ADCYAP1	HGNC:241
	CTF2P*	HGNC:33173
	N-phenylmaleimide	MESH:C028525
	Nanoparticles	MESH:D053758
	Nasopharynx	MESH:D009305
	Natriuretic Peptide*	FPLX:Natriuretic_peptide
	Neck Pain	MESH:D019547
	Neuralgia*	MESH:D009437
	Neuropeptides	MESH:D009479
	Neuropilins	MESH:D039921
	Neuroprostanes	MESH:D058632
	Neuroprotection	MESH:D000066829
	Nonylphenol*	CHEBI:CHEBI:79516
	Nuclear Proteins	MESH:D009687
	Nucleocapsid	MESH:D019251
	Nucleocapsid Proteins*	MESH:D019590
	Nucleoplasmins	MESH:D056489
	Nucleoproteins*	MESH:D009698
	Nucleus Pulposus	MESH:D000070614
	PNP	HGNC:7892
	Pneumonia, Necrotizing	MESH:D000071067
	Polyuria	MESH:D011141
	napabucasin	MESH:C000621033
	naphthalene	CHEBI:CHEBI:16482
	naproxen	CHEBI:CHEBI:7476
	nedaplatin	CHEBI:CHEBI:31898
	neopterin	CHEBI:CHEBI:28670
	nifedipine	CHEBI:CHEBI:7565
	nitroprusside*	CHEBI:CHEBI:7596
	nuclear periphery	GO:GO:0034399

Class level metrics:
--------------------
Grounding      

In [36]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1